<a href="https://colab.research.google.com/github/ValentinKarev/ML_for_DNK/blob/main/Seminar_1_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Спектральное представление речевого сигнала. Введение в цифровую обработку сигналов

Рассмотрим основы работы с речевыми сигналами с использованием библиотеки `torchaudio`

In [ ]:
from matplotlib import pyplot as plt
from IPython import display

import matplotlib
import torch
import torchaudio
from scipy.io import wavfile

import numpy as np

matplotlib.rc('xtick', labelsize=16)
matplotlib.rc('ytick', labelsize=16)

In [ ]:
wav, sr = torchaudio.load('data/LJ001-0001.wav')
print(f'Sample rate = {sr}, Wave = {wav}, Type: {wav.dtype}, Shape: {wav.shape}')

In [ ]:
def visualize_audio(wav: torch.Tensor, sr: int = 22050):
    # Усредняем при необходимости каналы
    if wav.dim() == 2:
        # Преобразуем в моно
        wav = wav.mean(dim=0)

    time_scale = torch.arange(wav.shape[0]) / sr

    plt.figure(figsize=(20, 5))
    plt.plot(time_scale, wav, alpha=.7, c='green')
    plt.grid()
    plt.xlabel('Time, seconds', size=20)
    plt.ylabel('Amplitude', size=20)
    plt.show()

    display.display(display.Audio(wav, rate=sr))

In [ ]:
visualize_audio(wav)

### Fast Fourier Transform

In [ ]:
n_fft = 1024
spectrum = torch.fft.rfft(wav, n=n_fft)
print(spectrum.dtype)

spectrogram = spectrum.abs().pow(2)
spectrogram_2 = torch.view_as_real(spectrum).norm(dim=-1).pow(2)

print(torch.allclose(spectrogram, spectrogram_2))

Изобразим спектр мощности (квадратов магнитуд) первых 1024 отсчетов нашего сигнала.

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(spectrogram.squeeze(), c='green')
plt.grid()
plt.xlabel('Frequency, Hz', size=20)
plt.ylabel('Magnitude$^2$', size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.show()

Применим оконную функцию Ханна

In [ ]:
window_size = n_fft
window = torch.hann_window(window_size)

plt.figure(figsize=(20, 5))
plt.plot(window, c='green')
plt.xticks(size=16)
plt.yticks(size=16)
plt.grid()
plt.show()

In [ ]:
clipped_wav = wav[:, :window_size]
windowed_clipped_wav = window * clipped_wav

fig, axes = plt.subplots(1, 2, figsize=(20, 5))

axes[0].plot(clipped_wav.squeeze(), c='green')
axes[0].set_title('Raw Audio', size=20)

axes[1].plot(windowed_clipped_wav.squeeze(), c='green')
axes[1].set_title('Windowed Audio', size=20)

for i in range(2):
    axes[i].grid()
    axes[i].set_xlabel('Time', size=20)
    axes[i].set_ylabel('Amplitude', size=20)

plt.show()

In [ ]:
spectrogram = torch.fft.rfft(clipped_wav).abs().pow(2)
windowed_spectrogram = torch.fft.rfft(windowed_clipped_wav).abs().pow(2)

fig, axes = plt.subplots(1, 2, figsize=(20, 5))

axes[0].plot(spectrogram.squeeze(), c='green')
axes[0].set_title('Spectrogram of Raw Audio', size=20)

axes[1].plot(windowed_spectrogram.squeeze(), c='green')
axes[1].set_title('Spectrogram of Windowed Audio', size=20)

for i in range(2):
    axes[i].grid()
    axes[i].set_xlabel('Frequency (Hz)', size=20)

plt.show()

STFT (short-time Fourier transform) для всего сигнала

In [ ]:
n_fft = 1024
window_size = n_fft
hop_size = 256
window = torch.hann_window(n_fft)

spectrum = torch.stft(
    wav,
    n_fft=n_fft,
    hop_length=hop_size,
    win_length=window_size,
    window=window,
    center=False,
    onesided=True,
    return_complex=True
)

print(f'Shape of spectrum: {spectrum.shape}, Type of spectrum: {spectrum.dtype}')

In [ ]:
spectrogram_real_imag = torch.view_as_real(spectrum)
print(f'Shape of spectrum: {spectrogram_real_imag.shape}, Type of spectrum: {spectrogram_real_imag.dtype}')

In [ ]:
spectrogram = spectrogram_real_imag.norm(dim=-1).pow(2)
print(f'Shape of spectrum: {spectrogram.shape}, Type of spectrum: {spectrogram.dtype}')

In [ ]:
plt.figure(figsize=(20, 5))
plt.imshow(spectrogram.squeeze().log()) # logarithmic scale
plt.xlabel('Time', size=20)
plt.ylabel('Frequency (Hz)', size=20)
plt.show()

Мел-шкала частот

In [ ]:
mel_scaler = torchaudio.transforms.MelScale(
    n_mels=80,
    sample_rate=22_050,
    n_stft=n_fft // 2 + 1
)

print(f'Shape of mel_scaler: {mel_scaler.fb.shape}')
print(mel_scaler.fb.T)

In [ ]:
plt.figure(figsize=(20, 5))
plt.imshow(mel_scaler.fb.T)
plt.xlabel('Hertz Scale', size=20)
plt.ylabel('Mels Scale', size=20)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
mel_spectrogram = mel_scaler(spectrogram)
print(f'Shape of mel spectrogram: {mel_spectrogram.shape}')

In [ ]:
plt.figure(figsize=(20, 5))
plt.imshow(mel_spectrogram.squeeze().log())
plt.xlabel('Time', size=20)
plt.ylabel('Mels', size=20)
plt.show()

In [ ]:
# torchaudio.transforms.MelSpectrogram

## Аугментации речи

Рассмотрим основные виды аугментаций, применяемых к речевым сигналам. В данном примере рассматриваются только примеры на основе библиотек `torchaudio` и `librosa`

### 1. Гауссов шум

In [ ]:
from torch import distributions

noiser = distributions.Normal(0, 0.05)
augumented_wav_1 = wav + noiser.sample(wav.size())

visualize_audio(augumented_wav_1)

### 2. Растяжение/сжатие по временной шкале

In [ ]:
# naive approach (don't do this!)

simple_stretch = wav[:, ::2]
visualize_audio(simple_stretch)

In [ ]:
# first approach

import librosa

augumented_wav_2 = librosa.effects.time_stretch(wav.squeeze().numpy(), rate=2)
augumented_wav_2 = torch.from_numpy(augumented_wav_2)
visualize_audio(augumented_wav_2)

In [ ]:
# second approach
n_fft = 400

stretcher = torch.nn.Sequential(
    torchaudio.transforms.Spectrogram(n_fft=n_fft, power=None),
    torchaudio.transforms.TimeStretch(n_freq = n_fft // 2 + 1, fixed_rate=2.0),
    torchaudio.transforms.InverseSpectrogram(n_fft=n_fft),
)

augumented_wav_2_v2 = stretcher(wav)
visualize_audio(augumented_wav_2_v2)

### Изменение частоты голоса (pitch shifting)

In [ ]:
augumented_wav_3 = librosa.effects.pitch_shift(wav.squeeze().numpy(), sr=sr, n_steps=-5)
augumented_wav_3 = torch.from_numpy(augumented_wav_3)

visualize_audio(augumented_wav_3)

### Изменение громкости

In [ ]:
voler = torchaudio.transforms.Vol(gain=0.2, gain_type='amplitude')
augumented_wav_4 = voler(wav)
visualize_audio(augumented_wav_4)

### Добавление реверберации

Производится за счет свертки импульсной характеристики помещения с аудиосигналом. Подробнее можно почитать здесь:

1) https://www.acousticalsurfaces.com/acoustic_IOI/reverberation.htm
2) https://www.sonic-shield.com/echo-vs-reverberation
3) https://en.wikipedia.org/wiki/Convolution_reverb
4) https://danielpovey.com/files/2017_icassp_reverberation.pdf

In [ ]:
rir, rir_sr = torchaudio.load('data/rirs/greathall.wav')
visualize_audio(rir, rir_sr)

In [ ]:
def simulate(audio: torch.Tensor, rir: torch.Tensor):
    left_pad = right_pad = rir.shape[-1] - 1

    # Разворачиваем ядро свертки, так как torch.conv вычисляет кросс-корреляцию, а не свертку
    flipped_rir = rir.squeeze().flip(0)

    audio = torch.nn.functional.pad(audio, [left_pad, right_pad]).view(1, 1, -1)
    convolved_audio = torch.conv1d(audio, flipped_rir.view(1, 1, -1)).squeeze()

    # peak normalization
    if convolved_audio.abs().max() > 1:
        convolved_audio /= convolved_audio.abs().max()

    return convolved_audio

In [ ]:
augumented_wav_5 = simulate(wav, rir)
visualize_audio(augumented_wav_5)

### Добавление фонового шума

Нужно, если мы хотим добиться устойчивости нашей модели к шуму, сопровождающему записанный аудиосигнал. Можно почитать:

1) https://medium.com/analytics-vidhya/adding-noise-to-audio-clips-5d8cee24ccb8
2) https://arxiv.org/pdf/1808.00563.pdf (параграф 3.1)

In [ ]:
noise, noise_sr = torchaudio.load('data/noises/zavod.wav')
print(noise.shape, noise_sr)
visualize_audio(noise, noise_sr)

In [ ]:
resampled_noise = torchaudio.transforms.Resample(noise_sr, sr)(noise)
visualize_audio(resampled_noise, sr)

In [ ]:
background_noise = resampled_noise[..., sr:sr + wav.shape[-1]]
visualize_audio(background_noise, sr)

In [ ]:
noize_level = torch.Tensor([0])  # [0, 40]

noize_energy = torch.norm(background_noise)
audio_energy = torch.norm(wav)

alpha = (audio_energy / noize_energy) * torch.pow(10, -noize_level / 20)

augumented_wav_6 = wav + alpha * background_noise

# В некоторых случаях сигнал может выйти за пределы [-1, 1]
augumented_wav_6 = torch.clamp(augumented_wav_6, -1, 1)

visualize_audio(augumented_wav_6)

### SpecAug (частотно-временное маскрование, "вырезы" в спектре)

Применяются к мел-спектрограмме. `SpecAug` - это аугментации, применяемые к ней так, как будто она представляет собой просто изображение.

Детали есть в работе https://arxiv.org/pdf/1904.08779.pdf

In [ ]:
mel_spectrogramer = torchaudio.transforms.MelSpectrogram(
    sample_rate=22050,
    n_fft=1024,
    win_length=1024,
    hop_length=256,
    n_mels=80,
)

mel_spectrogram = mel_spectrogramer(wav)
log_mel_spectrogram = torch.log(mel_spectrogram).squeeze()
plt.figure(figsize=(20, 5))
plt.imshow(log_mel_spectrogram)
plt.show()

`torchaudio.transforms.FrequencyMasking` добавляет горзонтальную полосу, а `torchaudio.transforms.TimeMasking` вертикальную.

Это не единственный возможный способ применять `SpecAug`.

In [ ]:
specaug = torch.nn.Sequential(
    torchaudio.transforms.FrequencyMasking(20),
    torchaudio.transforms.TimeMasking(100),
)

augumented_log_mel_spectrogram = specaug(log_mel_spectrogram)

plt.figure(figsize=(20, 5))
plt.imshow(augumented_log_mel_spectrogram)
plt.show()

### Задания на самостоятельную работу

1. Изучите преобразование `torchaudio.transforms.Preemphasis`. Оно часто применяется для предварительной обработки речевого сигнала в задаче верификации дикторов. Постройте STFT-спектры аудиосигнала LJ001-0001.wav без применения и с применением этого преобразования. Что в них изменяется?
2. Попробуйте добавить фоновый шум с помощью преобразования `torchaudio.transforms.AddNoise`. Постройте STFT-спектр фонового шума и зашумленного сигнала.
3. Рассмотрите преобразование `torchaudio.transforms.Convolve`. Примените с его помощью импульсную характеристику помещения к нашему аудиосигналу. Постройте STFT-спектр полученного сигнала. Что в нем изменилось по сравнению с оригинальным сигналом, как можно охарактеризовать эти изменения?